In [ ]:
from boardlaw.grid import *

In [ ]:
vast.jittenate(local=True)
launch()
while not jittens.finished():
    jittens.refresh()
    time.sleep(5)

In [ ]:
!rm -rf .jittens

In [ ]:
import jittens
jittens.jobs.jobs()